# Description of the project

Online store "Wikishop" launches a new service.
<br>Now users can edit and complete product descriptions, just like in wiki communities.
<br>That is, clients propose their edits and comment on the changes of others.
<br>The store needs a tool that will look for toxic comments and send them for moderation.
It is necessary to train the model to classify comments into positive and negative.
<br>There is a data set with markup on the toxicity of edits.
<br>The model must have a quality metric F1 of at least 0.75.

# Description of data

The data is in the `/datasets/toxic_comments.csv` file.
<br>The `text` column contains the text of the comment, and `toxic` is the target feature.

# Action plan

1. Download and prepare data.
2. Train different models.
3. Draw conclusions.

# Loading data

In [3]:
import pandas as pd
import numpy as np
import copy

import re
import nltk
from nltk.stem import SnowballStemmer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from tqdm._tqdm_notebook import tqdm_notebook
from tqdm import notebook

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

import torch
import transformers
from transformers import BertConfig, TFBertForSequenceClassification

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns',100)

In [4]:
df = pd.read_csv('/datasets/toxic_comments.csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


# Data preparation and model training without using BERT

## Data preparation

In [5]:
eng_stemmer = SnowballStemmer('english')

def stem_clear(text):
    stem_text = eng_stemmer.stem(text)   
    stem_text = re.sub(r'[^a-zA-Z]', ' ', stem_text)   
    return " ".join(stem_text.split())

In [6]:
tqdm_notebook.pandas()

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [7]:
df['stem_text'] = df['text'].progress_apply(stem_clear)
df.head()

,text,toxic,stem_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [ ]:
features = df['stem_text'].values.astype('U')
target = df['toxic']

In [1]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

In [8]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

print(features.shape)
print(target.shape)
print()

print(features_train.shape)
print(target_train.shape)
print()

print(features_test.shape)
print(target_test.shape)

(159571,)
(159571,)

(119678,)
(119678,)

(39893,)
(39893,)


In [9]:
%%time
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)
count_tf_idf

Wall time: 8.87 s


TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

## Model training

### Logistic regression

In [10]:
%%time
model = LogisticRegression(class_weight='balanced')
model.fit(tf_idf_train, target_train)

Wall time: 5.01 s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced')

In [11]:
predictions = model.predict(tf_idf_train)
print('F1 Logistic Regression on train set:', f1_score(target_train, predictions))

predictions = model.predict(tf_idf_test)
print('F1 Logistic Regression on test set:', f1_score(target_test, predictions))

F1 логистической регрессии на тренировочной выборке: 0.8417814344482203
F1 логистической регрессии на тестовой выборке: 0.7581101676464186


In [12]:
dummy_clf = DummyClassifier(strategy='stratified', random_state=12345)
dummy_clf.fit(tf_idf_train, target_train)
predictions = dummy_clf.predict(tf_idf_test)
print('F1 dummy on test set:', f1_score(target_test, predictions))

F1 дамми на тестовой выборке: 0.1091392007726669


### LightGBM

In [13]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'metric': 'f1',
    'learning_rate': 0.005,
    'verbose': 0,
    "max_depth": 20,
    "num_iterations": 5000,
    "n_estimators": 1000
}

In [14]:
%%time
gbm = lgb.LGBMClassifier(**hyper_params)
gbm.fit(tf_idf_train, target_train, verbose=0)
gbm.best_score_

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Wall time: 17min 51s


defaultdict(collections.OrderedDict, {})

In [15]:
predictions = gbm.predict(tf_idf_train)
print('F1 LightGBM on the train set:', f1_score(target_train, predictions))

predictions = gbm.predict(tf_idf_test)
print('F1 LightGBM on the test set:', f1_score(target_test, predictions))

F1 LightGBM на тренировочной выборке: 0.8282202833286424
F1 LightGBM на тестовой выборке: 0.7590004235493434


I did not use a decision tree and a random forest, since they require vectors of the same length, which in this case is not very good. I tried to train the model, but even the decision tree takes a very long time to train.

## Conclusion

As can be seen from the results on the test dataset, LightGBM takes first place, and logistic regression is in second place.
<br>But logistic regression wins in execution time.
<br>If the class balancing parameter is not used in the logistic regression, then F1 is less than 0.75.
<br>I think LightGBM can show the result better, but then you need to change the hyperparameters, which will lead to an increase in execution time.

# Data preparation and model training using BERT

## Data preparation

In [3]:
tqdm_notebook.pandas()

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
%%time
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')

tokenized = df['text'].progress_apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length = 512, truncation = True))

max_len = 0
for i in notebook.tqdm(tokenized.values):
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)


Wall time: 3min 56s


In [5]:
config = BertConfig.from_pretrained('bert-base-cased') 
model = transformers.BertModel(config=config)

In [1]:
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(
    attention_mask[batch_size*i:batch_size*(i+1)])
    
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [ ]:
features = np.concatenate(embeddings)
target = df['target']

In [ ]:
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.25)

## Neural network training

In [ ]:
%%time
model.fit(train_features, train_target)

In [ ]:
predictions = model.predict(tf_idf_train)
print('F1 BERT on the train set:', f1_score(target_train, predictions))

predictions = model.predict(tf_idf_test)
print('F1 BERT on the test set:', f1_score(target_test, predictions))